## Amazon Price Tracker with Email Alert System

As part of a course assignment, I developed this Python program to track product prices on Amazon and automatically send an email alert when the price drops below a predefined target. The project leverages web scraping techniques to extract product information such as the current price and title, compares the price to my set threshold, and sends me a notification when a deal is available.

This automation not only serves as a practical tool for real-world price monitoring but also demonstrates the power of Python for handling web data and email integration. By building this system, I was able to explore key concepts like web scraping, data extraction, and email automation, all while solving a problem that many shoppers face.

It’s a project that combines both technical learning and practical application, showcasing how programming can simplify everyday tasks and enhance efficiency.

### Inmported Libraries

In [ ]:
import logging
import os
import re
import requests
import smtplib

- **Requests**: For fetching the HTML content of the Amazon product page.
- **BeautifulSoup (bs4)**: For parsing and scraping the product details like price and title from the HTML.
- **smtplib**: For sending an email alert when the price drops below the desired target.
- **dotenv**: To securely store sensitive information such as email credentials and target price in a `.env` file.

In [ ]:
from bs4 import BeautifulSoup
from dotenv import load_dotenv

### Technical Overview

1. **Web Scraping**:
   The script uses the `requests` library to send a GET request to the Amazon product page. The response contains the HTML content, which is then parsed using the `BeautifulSoup` library to extract relevant data. Specifically, we look for:
   - Product title: Found in the HTML element with the `id="productTitle"`.
   - Product price: Extracted from the element with the `class="a-offscreen"`.

2. **Price Comparison**:
   The scraped price is converted from text to a float and compared with a predefined target price that is stored in an environment variable (`TARGET_PRICE`). If the price is below the target, the program proceeds to send an email.

3. **Email Alert**:
   The email is sent using Python’s `smtplib` library, which connects to Gmail’s SMTP server (or another email provider’s server). The email credentials (username, password, and server details) are securely stored in the `.env` file.

4. **Error Handling**:
   The program includes error handling for:
   - Failed HTTP requests when fetching the product page.
   - Inability to find or extract price and title data from the HTML.
   - Errors during email login or sending the email.

The `.env` file is used to securely store the following:
- **SMTP_ADDRESS**: The address of the email SMTP server (e.g., smtp.gmail.com).
- **EMAIL_ADDRESS**: Your email address from which the alert will be sent.
- **EMAIL_PASSWORD**: The app-specific password for the email (if 2FA is enabled).
- **TARGET_PRICE**: The price that triggers the alert.

In [ ]:
# Creating a requirements.txt file

with open('requirements.txt', 'w') as f:
    f.write('requests\n')
    f.write('beautifulsoup4\n')
    f.write('python-dotenv\n')
    f.write('smtplib\n')

print("requirements.txt file has been created.")

### Sample `.env` file

The `.env` file stores sensitive information securely. Below is an example of how your `.env` file should look. You will need to create this file manually in your project folder or use the following code to automatically generate it.

```plaintext
SMTP_ADDRESS="smtp.example.com"
EMAIL_ADDRESS="your_email@example.com"
EMAIL_PASSWORD="your_app_specific_password"
TARGET_PRICE=100

### Web Scraping and Email Alert System

The following code implements the price tracker and email alert system. The script performs the following tasks:

1. **Fetch Product Page**: Using the `requests` library, it sends a GET request to the Amazon product page.
2. **Extract Product Data**: It uses `BeautifulSoup` to scrape the product title and price from the page content.
3. **Price Comparison**: The current product price is compared to a target price (stored in the `.env` file).
4. **Send Email Alert**: If the price is below the target, it sends an email alert using the `smtplib` library.

### Load Environment Variables and Set up Logging

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO)

- **load_dotenv()** loads environment variables from the .env file, which contains sensitive data like email credentials and the target price.
- **logging.basicConfig(level=logging.INFO)** sets up logging so that the program will log information (like product titles and prices) and errors during the execution.

### Define Amazon Product URL and Headers

In [ ]:
# URL of the Amazon product (hardcoded for this exercise)
url = "https://www.amazon.com/dp/B075CYMYK6?psc=1&ref_=cm_sw_r_cp_ud_ct_FM9M699VKHTT47YD50Q6"

# Set up headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
}

- The URL is hardcoded for the Amazon product we are tracking (since this is an exercise).
- The headers dictionary mimics a request from a web browser, ensuring the request is not blocked by the website’s security measures.

### Fetch the Amazon Product Page

In [ ]:
# Fetch the Amazon page
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an error for bad responses
except requests.exceptions.RequestException as e:
    logging.error(f"Error fetching the page: {e}")
    exit()

- This code sends an HTTP request to fetch the content of the Amazon product page using the requests library.
- **raise_for_status()** checks if there were any issues with the request (like a 404 or 500 error). If there is an issue, it raises an exception, which we handle in the except block.

### Parse the Product Information Using BeautifulSoup

In [ ]:
# Parse the page content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Try to extract product title and price
try:
    # Fetching the price using the appropriate class
    price_text = soup.find("span", class_="a-offscreen").get_text()
    price_match = re.search(r'\$([\d,\.]+)', price_text)
    
    if price_match:
        price_without_currency = price_match.group(1).replace(',', '')
        price_as_float = float(price_without_currency)
    else:
        logging.error("Price not found or not in the expected format.")
        exit()

    title = soup.find(id="productTitle").get_text().strip()
except AttributeError as e:
    logging.error(f"Error extracting data: {e}")
    exit()

- BeautifulSoup is used to parse the HTML content of the product page.
- We extract the product price by finding the element with the class a-offscreen (which holds the price) and use a regular expression to clean up the price format.
- The product title is extracted from the element with id="productTitle".
- We handle any AttributeError in case the structure of the page changes or if the elements we're looking for don’t exist.

### Compare Price with Target and Send Email

In [ ]:
logging.info(f"Title: {title}")
logging.info(f"Price: ${price_as_float}")

# Fetch the buy price from the .env file
BUY_PRICE = float(os.getenv("TARGET_PRICE"))

# If the product price is below the set buy price, send an email
if price_as_float < BUY_PRICE:
    message = f"{title} is on sale for ${price_as_float}!"
    
    try:
        # Setup email connection
        smtp_address = os.getenv("SMTP_ADDRESS")
        email_address = os.getenv("EMAIL_ADDRESS")
        email_password = os.getenv("EMAIL_PASSWORD")

        if not smtp_address or not email_address or not email_password:
            logging.error("Email environment variables are not set properly.")
            exit()

        # Send the email using smtplib
        with smtplib.SMTP(smtp_address, 587) as connection:
            connection.starttls()  # Secure the connection
            connection.login(email_address, email_password)
            connection.sendmail(
                from_addr=email_address,
                to_addrs=email_address,
                msg=f"Subject:Amazon Price Alert!\n\n{message}\n{url}".encode("utf-8")
            )
        logging.info("Email sent successfully.")
    except smtplib.SMTPException as e:
        logging.error(f"Error sending email: {e}")
else:
    logging.info(f"Price is still higher than the target price of ${BUY_PRICE}. Current price: ${price_as_float}.")

- The product title and price are logged for reference.
- The target price (BUY_PRICE) is fetched from the .env file. If the actual price (price_as_float) is below this threshold, an email alert is triggered.
- smtplib is used to send an email via an SMTP server (in this case, Gmail). If the credentials or the SMTP setup are incorrect, an error is logged.
- If the price is higher than the target price, it simply logs that no email was sent.

## Conclusion

This price tracking and email alert system automates the process of monitoring product prices on Amazon, making it easier to track price drops and receive notifications in real-time. It offers a practical way to ensure you're always aware of the best deals without having to manually check the prices yourself.

In this exercise, I gained valuable experience in:

- Web scraping with requests and BeautifulSoup to extract data from web pages.
- Handling exceptions and errors, which are common in real-world web scraping scenarios.
- Automating email notifications using the smtplib library.
- Managing sensitive information securely using the .env file and dotenv package.

For the sake of simplicity, I hardcoded the Amazon product URL since this is an exercise. However, the script can be easily modified to accept dynamic URLs, making it more versatile. By adjusting the TARGET_PRICE and URL, the system can be customized to track any product on Amazon.

This project could be further developed by adding features like price history logging, tracking multiple products at once, or extending the scraper to other online retailers, making it a strong foundation for more advanced applications.